In [ ]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 22.6 MB/s eta 0:00:00


In [ ]:
pip install -q google-colab-selenium

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import date
import requests
import json
from pandas import json_normalize

import pandas as pd

pd.set_option('display.max_columns', None)


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.support.ui import WebDriverWait
from google.colab import files
from selenium.webdriver.support import expected_conditions as EC


In [ ]:
import google_colab_selenium as gs
driver = gs.Chrome()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
driver.maximize_window()

In [ ]:
# Enter urls here
urls = """


""".split("\n")

urls = [a for a in urls if len(a) > 20]
urls

In [ ]:
driver.get(urls[0])

In [ ]:
# burası çerezleri kabul etme
try:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID,'onetrust-accept-btn-handler'))).click()
    print("cookies accepted")
except:
    print("cookies cannot accepted")

cookies accepted


In [ ]:
# Clicking somewhere blank
from selenium.webdriver.common.action_chains import ActionChains
actions = ActionChains(driver)
actions.move_by_offset(10, 10).click().perform()
time.sleep(2)


In [ ]:
def get_table(url_):
    final_dfs = []
    for url in url_:
        print(url)
        driver.get(url)
        title = driver.title
        try:
            see_more_button = driver.find_element(By.XPATH, '//div[contains(text(), "DAHA FAZLA SATICI GÖSTER")]')
            see_more_button.click()
        except:
            print("cannot click 'daha fazla satıcı göster button'")
        product_blocks = driver.find_elements(By.CSS_SELECTOR, '.pr-mc-w.gnr-cnt-br')
        products_info = []

        for block in product_blocks:
            product_info = {
                'seller_name': '',
                'seller_rating': '',
                'badges': [],
                'delivery_info': '',
                'offers': [],
                'price': '',
                'product_link': ''
            }

            try:
                seller_element = block.find_element(By.CSS_SELECTOR, '.seller-name-text')
                product_info['seller_name'] = seller_element.text
            except:
                product_info['seller_name'] = 'No seller'

            try:
                rating_element = block.find_element(By.CSS_SELECTOR, '.sl-pn')
                product_info['seller_rating'] = rating_element.text.strip('"')
            except:
                product_info['seller_rating'] = 'No rating'

            try:
                badges = block.find_elements(By.CSS_SELECTOR, '.merchant-badge-container img')
                product_info['badges'] = [badge.get_attribute('src') for badge in badges]
            except:
                print("rozet errorrr")
                product_info['badges'] = []

            try:
                delivery_info_elements = block.find_elements(By.CSS_SELECTOR, '.pr-dd strong')
                delivery_info_texts = [elem.text for elem in delivery_info_elements]
                product_info['delivery_info'] = " ".join(delivery_info_texts)
            except:
                product_info['delivery_info'] = 'No delivery info'

            try:
                offers_elements = block.find_elements(By.CSS_SELECTOR, '.pr-mb-pr')
                for offer in offers_elements:
                    description = offer.find_element(By.CSS_SELECTOR, '.pr-mb-pr-tx').text
                    product_info['offers'].append(description)
            except:
                product_info['offers'] = []

            try:
                price_element = block.find_element(By.CSS_SELECTOR, '.prc-dsc')
                product_info['price'] = price_element.text
            except:
                product_info['price'] = 'No price'

            try:
                link_element = block.find_element(By.CSS_SELECTOR, '.pr-om-1nk-btn')
                product_info['product_link'] = link_element.get_attribute('href')
            except:
                product_info['product_link'] = 'No link'

            products_info.append(product_info)

        def ff(val):
            ss = val.copy()

            for i in range(len(val["badges"])):
                ss["badge_" +str(i)] = val["badges"][i]

            for i in range(len(val["offers"])):
                ss["offer_" +str(i)] = val["offers"][i]
            return ss

        desired = ["seller_name","seller_rating","delivery_info","delivery_info","badge_0","badge_1","badge_2","offer_0","offer_1","offer_2","price"]
        df_list = []
        for vals in products_info:
            cols = [i for i in ff(vals).keys() if i in desired]
            aa = pd.DataFrame([ff(vals)[i] for i in ff(vals).keys() if i in desired]).T
            aa.columns = cols
            df_list.append(aa)

        df = pd.concat(df_list)
        # df["url"] = url_
        df["title"] = title
        final_dfs.append(df)
        time.sleep(2)

    return final_dfs



In [ ]:
ss = get_table(urls)
main_df = pd.concat(ss)

In [ ]:
badge_cols = [i for i in main_df.columns if "badge" in i]

if len(badge_cols) == 2:
    lll = []
    for i in main_df[badge_cols].fillna("nan").values:
        ll=[]
        if "basarili_satici" in " ".join(list(i)):
            ll.append("basarili satici")
        else:
            ll.append("")
        if "hizli-satici" in " ".join(list(i)):
            ll.append("hizli satici")
        else:
            ll.append("")
        lll.append(ll)
    main_df[badge_cols] = lll
if len(badge_cols) == 1:
    lll = []
    for i in main_df[badge_cols].fillna("nan").values:
        ll=[]
        if "basarili_satici" in " ".join(list(i)):
            ll.append("basarili satici")
        elif "hizli-satici" in " ".join(list(i)):
            ll.append("hizli satici")
        else:
            ll.append("")
        lll.append(ll)
    main_df[badge_cols] = lll
if len(badge_cols) == 3:
    lll = []
    for i in main_df[badge_cols].fillna("nan").values:
        ll=[]
        if "basarili_satici" in " ".join(list(i)):
            ll.append("basarili satici")
        else:
            ll.append("")
        if "hizli-satici" in " ".join(list(i)):
            ll.append("hizli satici")
        else:
            ll.append("")
        ll.append(i[2].split("/")[-1].split(".")[0])
        lll.append(ll)
    main_df[badge_cols] = lll

In [ ]:
offer_cols = [i for i in main_df.columns if "offer" in i]


if len(offer_cols) == 3:
    lll = []
    for i in main_df[offer_cols].fillna("nan").values:
        ll=[]
        if "Kurumsal Fatura" in " ".join(list(i)):
            ll.append("Kurumsal Fatura")
        else:
            ll.append("")
        if "Kargo Bedava" in " ".join(list(i)):
            ll.append("Kargo Bedava")
        else:
            ll.append("")
        d = [a for a in list(i) if a not in ["Kargo Bedava","Kurumsal Fatura"]]
        if d == ["nan"]:
            ll.append("")
        else:
            ll.append(d[0])

        lll.append(ll)
    main_df[offer_cols] = lll
if len(offer_cols) == 2:
    aa = main_df[offer_cols].fillna("nan")
    aa["offer_2"] = "nan"
    lll = []
    for i in aa.values:
        ll=[]
        if "Kurumsal Fatura" in " ".join(list(i)):
            ll.append("Kurumsal Fatura")
        else:
            ll.append("")
        if "Kargo Bedava" in " ".join(list(i)):
            ll.append("Kargo Bedava")
        else:
            ll.append("")
        d = [a for a in list(i) if a not in ["Kargo Bedava","Kurumsal Fatura"]]
        if d == ["nan"]:
            ll.append("")
        else:
            ll.append(d[0])

        lll.append(ll)
    main_df[offer_cols] = lll

if len(offer_cols) == 1:
    aa = main_df[offer_cols].fillna("nan")
    aa["offer_2"] = "nan"
    aa["offer_1"] = "nan"

    lll = []
    for i in aa.values:
        ll=[]
        if "Kurumsal Fatura" in " ".join(list(i)):
            ll.append("Kurumsal Fatura")
        else:
            ll.append("")
        if "Kargo Bedava" in " ".join(list(i)):
            ll.append("Kargo Bedava")
        else:
            ll.append("")
        d = [a for a in list(i) if a not in ["Kargo Bedava","Kurumsal Fatura"]]
        if d == ["nan"]:
            ll.append("")
        else:
            ll.append(d[0])

        lll.append(ll)
    main_df[offer_cols] = lll

In [ ]:
main_df.to_excel(f"buybox.xlsx")
files.download(f"buybox.xlsx")



In [ ]:
driver.quit()